In [ ]:
import time
import pandas as pd
import geopandas

In [ ]:
grid_features = geopandas.read_file('grid_features_4r.shp', dtype={'tile_ID':str})

In [ ]:
feature = grid_features[['pop', 'pop_den', 'road_den', 'inter_den', 'poi_den', 'price_k']]
tessel = grid_features[['geometry']]

In [ ]:
# tessel['centroid'] = tessel['geometry'].centroid

In [ ]:
didi_flow = pd.read_csv('didi_trip.csv')[['lng_start', 'lng_end', 'lat_start', 'lat_end']]

In [ ]:
#  def loc_match(lng, lat, tesselation):
#     match = 0
#     min_dist = 10
#     i = 0
#     while i < len(tessel):
#         lng_label = tessel['centroid'].x[i]
#         lat_label = tessel['centroid'].y[i]
#         dist_2 = pow((lng_label -lng), 2) + pow((lat_label - lat), 2)
#         if dist_2 < min_dist:
#             min_dist = dist_2
#             match = i
#         i += 1
#     return match

In [ ]:
didi_flow['match_start'] = -1
didi_flow['match_end'] = -1

def iter_match(didi):
    for id, item in didi.iterrows():
        didi['match_start'][id] = loc_match(item['lng_start'], item['lat_start'], tessel)
        didi['match_end'][id] = loc_match(item['lng_end'], item['lat_end'], tessel)

iter_match(didi_flow.head(3))

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import datetime,time

In [ ]:
length = len(tessel)
def single_flow(lng, lat, tesselation):
    point = Point(lng, lat)
    for i in range(length):
        polygon = Polygon(tessel['geometry'][i])
        if polygon.contains(point):
            return i
    return -2

In [ ]:
def didi_clean(data):
    for id, item in data.iterrows():
        data['match_start'][id] = single_flow(item['lng_start'], item['lat_start'], tessel)
        data['match_end'][id] = single_flow(item['lng_end'], item['lat_end'], tessel)
        if id % 1000 == 0:
            print(id, time.strftime("%d %H:%M:%S"), data['match_start'][id])

In [ ]:
path = '/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_'
def batch_clean(batch_size, remain, i):
    if remain.shape[0] > batch_size:
        deal_data = remain.head(batch_size)
        didi_clean(deal_data)
        deal_data.to_csv(str(path + str(i) + '.csv'), columns=['match_start', 'match_end'])
        i += 1
        batch_clean(batch_size, remain.tail(-batch_size), i)
    else:
        didi_clean(remain)
        remain.to_csv('./batch_clean/batch_last.csv')

In [ ]:
batch_clean(149297, didi_flow.tail(-149297 * 39), 40)

In [ ]:
df = pd.read_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_43.csv')
df2 = pd.read_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_44.csv')
df = df[['index', 'match_start', 'match_end']]
df2 = df2[['index','match_start', 'match_end']]

In [ ]:
df.to_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_43.csv', index=False)
df2.to_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_44.csv', index=False)

In [ ]:
import glob
import re

In [ ]:
csv_lst = ['/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_' + str(i+1) for i in range(0,44)]

In [ ]:
csv_lst[1]

In [ ]:
flow = pd.read_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_1.csv')

In [ ]:
for path in csv_lst:
    if path == '/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/batch_clean/batch_1':
        continue
    df = pd.read_csv(path + '.csv')
    flow = pd.concat([flow, df],join='inner',ignore_index=True)
flow.to_csv('flows.csv')

In [ ]:
flow = pd.read_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/flows.csv')

In [ ]:
flow 

In [ ]:
flow = flow.iloc[:, [2,3]]

In [ ]:
flow

In [ ]:
didi_trip = pd.read_csv('/Users/chizhang/Desktop/didi/deepgravity/data/didi_cd/didi_trip.csv')

In [ ]:
didi_trip = pd.concat([didi_trip, flow], axis=1)

In [ ]:
didi_trip

In [ ]:
didi_trip.to_csv('didi_trip_with_tessel.csv')